`updates from the last main_model_code.ipynb in master branch.`

`0.1 possibly some new libraries`

`2.1 I moved this code chunk from the bottom to here.`

`2.3 New code chunk -- this gets the Y label for the decoder`

`2.6 Major code change here to allow encoder option`

`3.2 New code chunk -- this calculates cosine similarity loss for decoder.`

`3.4, 3.5 compile_model() function is now customized as two compile_caps_model() and compile_cnn_model() for code claritys sake. corresponding loss functions and loss weights are updated`

`(you can delete the two existing code chunks which contains `def decoder_loss()` and compile_model())`

`3.6 fit_model() arguments have changed to allow for multiple model outputs, replace this whole chunk`

`4.0 'embed_dim' : embed_dim;  swtich 'use_decoder' to True to enable decoder`

`4.1 'embed_dim' : embed_dim`

`4.2 compile_model() now becomes compile_caps_model`

`4.3 compile_model() now becomes compile_cnn_model`

`4.4 New code chunk -- draw and compile CapsNet with decoder`

`4.5, 4.6 minor argument format change according to change described in 3.6`

`4.7 New code chunk to fit CapsNet with decoder`

### 0.0 Load Libraries

In [1]:
%load_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import tensorflow as tf
import keras as K
from keras import callbacks, optimizers
from keras import backend as KB
from keras.engine import Layer
from keras.layers import Activation
from keras.layers import LeakyReLU, Dense, Input, Embedding, Dropout, Reshape, Concatenate, MaxPooling1D, Flatten
from keras.layers import Bidirectional, GRU, Flatten, SpatialDropout1D, Conv1D
from keras.layers import Add
# from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Model
from keras.models import Sequential
from keras.utils import to_categorical
from common import vocabulary, utils

# capsule layers from Xifeng Guo 
# https://github.com/XifengGuo/CapsNet-Keras
from capsulelayers import CapsuleLayer, PrimaryCap, Length, Mask
import glove_helper

import time # !

/home/yeunghoman/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


### 0.1 

In [2]:
print("Tensorflow version:", tf.__version__)
print("Keras version:", K.__version__)

Tensorflow version: 1.4.1
Keras version: 2.1.5


### 0.2 

In [3]:
TRAIN_FILE = "../data/conll2003/eng.train"
DEV_FILE = "../data/conll2003/eng.testa"
TEST_FILE = "../data/conll2003/eng.testb"

## Data Class !

### 1.0 

In [4]:
# local untils

# timeit decorator
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print ('%r  %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000))
        return result
    return timed

### 1.1

In [5]:
def construct_embedding_matrix(embed_dim):
    """
    construct embedding matrix from GloVe 6Bn word data
    
    reuse glove_helper code from w266 
    
    Returns: an embedding matrix directly plugged into keras.layers.Embedding(weights=[embedding_matrix])
    """
    reload(glove_helper)
    hands = glove_helper.Hands(ndim=embed_dim)
    embedding_matrix = np.zeros((vocabData.vocab.size, embed_dim))

    for i in range(vocabData.vocab.size):
        word = vocabData.vocab.ids_to_words([i])[0]
        try:
            embedding_vector = hands.get_vector(word)
        except:
            embedding_vector = hands.get_vector("<unk>")
        embedding_matrix[i] = embedding_vector

    return embedding_matrix

### 1.2

In [6]:
class conll2003Data(object):
    """
    Keep track of data and processing operations for a single CoNLL2003 data file.
    """

    def __init__(self, filePath_train):
        """
        filePath(string): path to a CoNLL2003 raw data file for training the vocabulary
        """
        self.vocab = []
        self.posTags = []
        self.nerTags = []
        self.train_sentences = self.readFile(filePath_train)


    @timeit
    def readFile(self, filePath, canonicalize=True, verbose=False):
        """
        Read the conll2003 raw data file

        filename(string) - path to conll2003 file (train, test, etc.)
        
        Returns: a list of lists of lists corresponding to the words, pos tags, ner tags, capitalization
                 in each sentence

        """
        print ("----------------------------------------------------")
        print ("reading file from path", str(filePath))
        f = open(filePath)
        sentences = []
        sentence = []
        for line in f:
            if len(line) == 0 or line.startswith("-DOCSTART") or line[0] == '\n':
                if len(sentence) > 0:
                    sentences.append(sentence)
                    sentence = []
                continue
            
            # input format is [ word, pos tag, chunck tag, ner tag]
            # we are ignoring the chunck tag
            splits = line.strip().split(' ')
            if canonicalize:
                word = [utils.canonicalize_word(splits[0]), splits[1], splits[3], self.capitalizaion(splits[0])]
            else:
                word = [splits[0], splits[1], splits[3], capitalizaion(splits[0])]
            sentence.append( word)
        
        # don't forget the last sentence
        if len(sentence) > 0:
            sentences.append(sentence)
            sentence = []
        
        if verbose: 
            print ("number of sentences on file =",len(sentences))
            print ("first 5 sentences:")
            print (sentences[:5])

        return sentences
    
    
    def capitalizaion(self, word):
        """
        check capitalization info for a word
        return 'lowercase' for 'sfsd'
        return 'allCaps' for 'SFSD'
        return 'upperInitial' for 'Sfsd'
        return 'mixedCaps' for 'SfSd'
        return 'noinfo' for '$#%@#' or '12334'
        """
        alphas = [c.isalpha() for c in word] 
        if sum(alphas) != len(word):
            return 'noinfo'
        caps = [char.lower()==char for char in word]
        if sum(caps) == len(word):
            return 'lowercase'
        elif sum(caps) == 0:
            return 'allCaps'
        elif caps[0] == False and sum(caps) == len(word)-1:
            return 'upperInitial'
        elif 0 < sum(caps) < len(word):
            return 'mixedCaps'
        else:
            return 'noinfo'    
    
    @timeit
    def buildVocab(self, vocabSize=None, verbose=False, return_vocab_objects=False):
        """
        Builds the vocabulary based on the initial data file
        
        vocabSize(int, default: None-all words) - max number of words to use for vocabulary
                                                  (only used for training)
        verbose(boolean, default: False)        - print extra info
        """    	
        print ("----------------------------------------------------")
        print ("building vocabulary from TRAINING data...")

        flatData = [w for w in zip(*utils.flatten(self.train_sentences))]

        # remember these vocabs will have the <s>, </s>, and <unk> tags in there
        # sizes need to be interpreted "-3" - consider replacing...
        self.vocab = vocabulary.Vocabulary( flatData[0], size=vocabSize)
        self.posTags = vocabulary.Vocabulary( flatData[1])
        self.nerTags = vocabulary.Vocabulary( flatData[2])
        self.capitalTags = vocabulary.Vocabulary(flatData[3])

        if verbose:
            print ("vocabulary for words, posTags, nerTags built and stored in object")
            print ("vocab size =", vocabSize)
            print ("10 sampled words from vocabulary\n", list(self.vocab.wordset)[:10], "\n")
            print ("number of unique pos Tags in training =", self.posTags.size)
            print ("all posTags used\n", list(self.posTags.wordset), "\n")
            print ("number of unique NER tags in training =", self.nerTags.size)
            print ("all nerTags for prediction", list(self.nerTags.wordset), "\n")
            print ("number of unique capitalization tags in training =", self.capitalTags.size)
            print ('all capitalTags for prediction', list(self.capitalTags.wordset), "\n")

        if return_vocab_objects:
            return self.vocab, self.posTags, self.nerTags, self.capitalTags


    @timeit
    def formatWindowedData(self, sentences, windowLength=9, verbose=False):
        """
        Format the raw data by blocking it into context windows of a fixed length corresponding 
        to the single target NER tag of the central word.
        Make sure to call buildVocab first.
        
        sentences(list of lists of lists) - raw data from the CoNLL2003 dataset
        windowLength(int, default: 9)     - The length of the context window
                    NOTE - windowLength must be odd to have a central word. If itsn't, 1 will be added.
        verbose(boolean, default: False)  - print extra info
        
        Returns: 4 numpy arrays: vocabulary training data windowed and converted to IDs, 
                                 POS tags windowed and converted to IDs,
                                 Capitalization info windowed and converted into IDs,
                                 NER label tags converted to IDs
        """

        print ("----------------------------------------------------")
        print ("formatting sentences into input windows...")

        if windowLength % 2 == 0 or windowLength == 1:
            raise ValueError("window Length must be an odd number and greater than one.")
    
        pads = windowLength // 2

        # we have a list of lists (sentences) of lists ([word, posTag, nerTag])
        # parse through, pad each sentence with pads open and close tags, then convert to IDs
        vocabIDs = [ self.vocab.words_to_ids( ["<s>"] * pads + [word[0] for word in sent] + ["</s>"] * pads) \
                     for sent in sentences]
        posIDs = [ self.posTags.words_to_ids( ["<s>"] * pads + [word[1] for word in sent] + ["</s>"] * pads) \
                   for sent in sentences]
        capitalIDs = [self.capitalTags.words_to_ids(["<s>"]*pads + [word[3] for word in sent] + ["</s>"]*pads) \
                     for sent in sentences]
        nerIDs = [ self.nerTags.words_to_ids( ["<s>"] * pads + [word[2] for word in sent] + ["</s>"] * pads) \
                   for sent in sentences]
        
        if verbose: 
            print ("STEP 1/2 -- PADDING")
            print ("all sentences padded with {} pads to either end".format(pads))
            print ("vocab idx for first 5 sentences:\n", vocabIDs[:5], "\n")
            print ("pos idx for first 5 sentences:\n", posIDs[:5], "\n")
            print ("ner idx for first 5 sentences:\n", nerIDs[:5], "\n")
            print ("capitalization idx for first 5 sentences: \n", capitalIDs[:5], "\n")
            print ("number of sentences = {}".format(len(vocabIDs)), "\n")

        assert(len(vocabIDs) == len(posIDs) and len(posIDs) == len(nerIDs) == len(capitalIDs))

        if verbose: 
            print ("STEP 2/2 -- WINDOWING")

        # build the data to train on by sliding the window across each sentence
        # at this point, all 3 lists are the same size, so we can run through them all at once
        featsVocab, featsPOS, featsNER, featsCAPITAL = [], [], [], []
        for sentID in range( len(vocabIDs)):
            sent = vocabIDs[sentID]
            sentPOS = posIDs[sentID]
            sentNER = nerIDs[sentID]
            sentCAPITAL = capitalIDs[sentID]
            
            for ID in range( len(sent) - windowLength + 1):
                featsVocab.append( sent[ID:ID + windowLength])
                featsPOS.append( sentPOS[ID:ID + windowLength])
                featsCAPITAL.append(sentCAPITAL[ID:ID + windowLength])
                featsNER.append( sentNER[ID + windowLength // 2])
                
        if verbose:
            print ("sample windows:")
            for i in range(3):
                print ("Vocab for window {}".format(i))
                print (featsVocab[i])
                print (self.vocab.ids_to_words(featsVocab[i]))
                print ("PoS tags for window {}".format(i))
                print (featsPOS[i])
                print (self.posTags.ids_to_words(featsPOS[i]))
                print ("Capitalization tags for window {}".format(i))
                print (featsCAPITAL[i])
                print (self.capitalTags.ids_to_words(featsCAPITAL[i]))
                print ("NER tags for center word")
                print (featsNER[i])
                print (self.nerTags.ids_to_words([featsNER[i]]),"\n")
                
            print ("rows of vocab features = {}".format(len(featsVocab)))
            print ("rows of PoS features = {}".format(len(featsVocab)))
            print ("rows of Capitalization features = {}".format(len(featsCAPITAL)))
            print ("rows of NER features = {}".format(len(featsNER)))
            
            print ("numpy feature arrays are returned")

        assert(len(featsVocab) == len(featsVocab) == len(featsNER) == len(featsCAPITAL))
        return np.array(featsVocab), np.array(featsPOS), np.array(featsCAPITAL), np.array(featsNER)
    

## Load the Data

### 2.0

In [7]:


windowLength = 9
testNumSents = 5000

# Use training set to build vocab here
vocabData = conll2003Data(TRAIN_FILE)
vocabData.buildVocab( vocabSize=20000)

# Format training data
trainX, trainX_pos, trainX_capitals, trainY  = vocabData.formatWindowedData( vocabData.train_sentences, 
                                                  windowLength=windowLength,
                                                  verbose=False)

# read in dev data
devSents = vocabData.readFile( DEV_FILE)
devX, devX_pos, devX_capitals, devY = vocabData.formatWindowedData( devSents, 
                                              windowLength=windowLength,
                                              verbose=False)

# read in the test data
testSents = vocabData.readFile( TEST_FILE)
testX, testX_pos, testX_capitals, testY = vocabData.formatWindowedData( testSents, 
                                                windowLength=windowLength,
                                                verbose=False)

----------------------------------------------------
reading file from path ../data/conll2003/eng.train
'readFile'  1336.43 ms
----------------------------------------------------
building vocabulary from TRAINING data...
'buildVocab'  1104.56 ms
----------------------------------------------------
formatting sentences into input windows...
'formatWindowedData'  2017.61 ms
----------------------------------------------------
reading file from path ../data/conll2003/eng.testa
'readFile'  313.65 ms
----------------------------------------------------
formatting sentences into input windows...
'formatWindowedData'  528.49 ms
----------------------------------------------------
reading file from path ../data/conll2003/eng.testb
'readFile'  335.59 ms
----------------------------------------------------
formatting sentences into input windows...
'formatWindowedData'  628.77 ms


### 2.1

In [8]:
# !
embed_dim = 50
# Load GloVe embedding matrix
embedding_matrix = construct_embedding_matrix(embed_dim)

Loading vectors from data/glove/glove.6B.zip
Parsing file: data/glove/glove.6B.zip:glove.6B.50d.txt
Found 400,000 words.
Parsing vectors... Done! (W.shape = (400003, 50))


### 2.2

In [9]:
# Get Y

# encoding 1-hot for ner targets
trainY_cat = to_categorical(trainY.astype('float32'))
devY_cat = to_categorical(devY.astype('float32'), num_classes=trainY_cat.shape[1])
testY_cat = to_categorical(testY.astype('float32'), num_classes=trainY_cat.shape[1])

trainY_cat = np.array(list(map( lambda i: np.array(i[3:], dtype=np.float), trainY_cat)), dtype=np.float)
devY_cat = np.array(list(map( lambda i: np.array(i[3:], dtype=np.float), devY_cat)), dtype=np.float)
testY_cat = np.array(list(map( lambda i: np.array(i[3:], dtype=np.float), testY_cat)), dtype=np.float)

### DUMMY NOW

In [10]:
trainY

array([5, 3, 7, ..., 3, 5, 3])

In [11]:
# gold label
trainY_cat

array([[ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [15]:
trainY_cat.shape

(203621, 8)

In [13]:
# create dummy prediction labels
np.random.seed(0)
shuffle = np.random.permutation(np.arange(trainY_cat.shape[0]))
trainY_cat_pred = trainY_cat[shuffle]

In [14]:
trainY_cat_pred

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [16]:
trainY_cat_pred.shape

(203621, 8)

In [12]:
np.argmax(trainY_cat, axis=1)

array([2, 0, 4, ..., 0, 2, 0])

In [15]:
np.argmax(trainY_cat, axis=1) + 3

array([5, 3, 7, ..., 3, 5, 3])

In [ ]:
def calc_cross_entropy_per_row(y_true_row, y_pred_row):
    """
    Returns: a real number. cross-entropy for one single prediction
    """

In [ ]:
def print_worst_predictions(y_true, y_pred, k=5):
    """
    print worst k predictions by cross_entropy loss
    """
    

In [72]:
def get_precision(y_true, y_pred):
    """
    compute precision = correct entities / predicted entities
    
    Arguments:
        y_true : trainY/devY/testY. an array of shape(?,). Each value correspond to the ner tag for a word
        y_pred : model prediction. same shape and format as y_true
    
    Returns:
        precision : real number
    """
    model_entities_filter = (y_pred != 3).astype("int") # of the words our model say has a NER class
    precision_correct_entities = (y_pred[np.where(model_entities_filter)] == y_true[np.where(model_entities_filter)]).astype("int")
    precision = np.sum(precision_correct_entities)/np.sum(model_entities_filter)
    return precision

In [73]:
def get_recall(y_true, y_pred):
    """
    compute recall = correct entities / all gold entities
    
    Arguments:
        y_true : trainY/devY/testY. an array of shape(?,). Each value correspond to the ner tag for a word
        y_pred : model prediction. same shape and format as y_true
    
    Returns:
        recall : real number
    """
    true_entities_filter = (y_true != 3).astype("int") # of the words that truly has a NER class
    recall_correct_entities = (y_pred[np.where(true_entities_filter)] == y_true[np.where(true_entities_filter)]).astype("int")
    recall = np.sum(recall_correct_entities)/np.sum(true_entities_filter)
    return recall

In [74]:
def convert_raw_y_pred(raw_y_pred):
    """
    Arguments:
        raw_y_pred : raw predictions generated from model.predict() method. \
        this is a 2D matrix of shape (?, number of NER classes). Each row correspond to one 1-hot NER vector
    
    Returns:
        an array of shape (?,). Each value correspond to the predicted ner tag for a word.\
        You can convert this array back to NER tags by vocabData.ner_vocab.ids_to_words()
    """
    return np.argmax(raw_y_pred, axis=1) + 3

In [75]:
def get_f1(y_true, y_pred):
    precision = get_precision(y_true, y_pred)
    recall = get_recall(y_true, y_pred)
    f1 = 2*(precision*recall)/(precision + recall)
    return (precision, recall, f1)

# !!
assert(get_f1(y_true=np.array([5,7,3,4]), y_pred=np.array([[1,0,0,0,0],[0,0,1,0,0],[1,0,0,0,0],[0,1,0,0,0]])) == (1/2,1/3,0.4))

In [ ]:
# our model hallucinated that there's a NER class
def get_ner_hallucination(y_true, raw_y_pred):

In [ ]:
# our model missed a NER class
def get_missed_ner(y_true, raw_y_pred):

In [ ]:
# ner exist in both model pred and gold 
# model got right/wrong
def get_missed_ner(y_true, raw_y_pred):

In [ ]:
# construct dumb baseline

In [ ]:
# dissect by true ner tag
# see mistakes

### 2.3

In [10]:
# !
# Get decoder Y -- 50 dim embedding of center word

train_decoderY = embedding_matrix[trainX[:,4]]
dev_decoderY = embedding_matrix[devX[:,4]]
test_decoderY = embedding_matrix[testX[:,4]]

### 2.4

In [11]:
# Get X pos tags

# encoding 1-hot for pos tags
trainX_pos_cat = to_categorical(trainX_pos.astype('float32'))
devX_pos_cat = to_categorical(devX_pos.astype('float32'), num_classes=trainX_pos_cat.shape[2]) 
testX_pos_cat = to_categorical(testX_pos.astype('float32'), num_classes=trainX_pos_cat.shape[2])

trainX_pos_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), trainX_pos_cat)), dtype=np.float)
devX_pos_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), devX_pos_cat)), dtype=np.float)
testX_pos_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), testX_pos_cat)), dtype=np.float)

### 2.5

In [12]:
# Get X capitlization 

# encoding 1-hot for capitalization info  ("allCaps", "upperInitial", "lowercase", "mixedCaps", "noinfo")
trainX_capitals_cat = to_categorical(trainX_capitals.astype('float32'))
devX_capitals_cat = to_categorical(devX_capitals.astype('float32'), num_classes=trainX_capitals_cat.shape[2]) 
testX_capitals_cat = to_categorical(testX_capitals.astype('float32'), num_classes=trainX_capitals_cat.shape[2])

trainX_capitals_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), trainX_capitals_cat)), dtype=np.float)
devX_capitals_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), devX_capitals_cat)), dtype=np.float)
testX_capitals_cat = np.array(list(map( lambda i: np.array(i[:,3:], dtype=np.float), testX_capitals_cat)), dtype=np.float)

## Build the model

### 3.0

In [13]:
def draw_capsnet_model(hyper_param, embedding_matrix=None, verbose=True):
    """
    Input: hyper parameters dictionary
    
    Construct:
        input layers : x , x_pos(o), x_captialization(o)
        embedding matrix : use_glove or randomly initialize
        conv1 : first convolution layer
        primarycaps : conv2 and squash function applied
        ner_caps : make 8 ner capsules of specified dim
        out_pred : calc length of 8 ner capsules as 8 prob. predictions over 8 ner classes
    
    Returns: 
        if decoding/reconstruction disabled --> a single keras.models.Model object
        if decoding/reconstruction enabled --> three keras.models.Model objects
    """

    # input layer(s)
    x = Input(shape=(hyper_param['maxlen'],), name='x')
    if hyper_param['use_pos_tags'] : 
        x_pos = Input(shape=(hyper_param['maxlen'],hyper_param['poslen']), name='x_pos')
    if hyper_param['use_capitalization_info'] : 
        x_capital = Input(shape=(hyper_param['maxlen'], hyper_param['capitallen']), name='x_capital') 
    
    # embedding matrix
    if hyper_param['use_glove']:
        embed = Embedding(hyper_param['max_features'], hyper_param['embed_dim'], weights=[embedding_matrix],\
                          input_length=hyper_param['maxlen'], trainable=hyper_param['allow_glove_retrain'])(x)
    else:
        embed = Embedding(hyper_param['max_features'], hyper_param['embed_dim'], input_length=hyper_param['maxlen'],\
                          embeddings_initializer="random_uniform" )(x)

    # concat embeddings with additional features
    if hyper_param['use_pos_tags'] and hyper_param['use_capitalization_info'] : 
        embed = Concatenate(axis=-1)([embed, x_pos, x_capital])
    elif hyper_param['use_pos_tags'] and (not hyper_param['use_capitalization_info']) :
        embed = Concatenate(axis=-1)([embed, x_pos])
    elif (not hyper_param['use_pos_tags']) and hyper_param['use_capitalization_info'] :
        embed = Concatenate(axis=-1)([embed, x_capital])    
    else :
        embed = embed
    
    # feed embeddings into conv1
    conv1 = Conv1D( filters=hyper_param['conv1_filters'], \
                   kernel_size=hyper_param['conv1_kernel_size'],\
                   strides=hyper_param['conv1_strides'], \
                   padding=hyper_param['conv1_padding'],\
                   activation='relu', name='conv1')(embed)

    # make primary capsules
    primarycaps = PrimaryCap(embed, \
                             dim_capsule=hyper_param['primarycaps_dim_capsule'],\
                             n_channels=hyper_param['primarycaps_n_channels'],\
                             kernel_size=hyper_param['primarycaps_kernel_size'], \
                             strides=hyper_param['primarycaps_strides'], \
                             padding=hyper_param['primarycaps_padding'])

    # make ner capsules
    ner_caps = CapsuleLayer(num_capsule=hyper_param['ner_classes'], \
                            dim_capsule=hyper_param['ner_capsule_dim'], \
                            routings=hyper_param['num_dynamic_routing_passes'], \
                            name='nercaps')(primarycaps)

    # replace each ner capsuel with its length
    out_pred = Length(name='out_pred')(ner_caps)


    if verbose:
        print ("x", x.get_shape())
        if hyper_param['use_pos_tags'] : print ("x_pos", x_pos.get_shape())
        if hyper_param['use_capitalization_info'] : print ("x_capital", x_capital.get_shape())
        print ("embed", embed.get_shape())
        print ("conv1", conv1.get_shape())
        print ("primarycaps", primarycaps.get_shape())   
        print ("ner_caps", ner_caps.get_shape())
        print ("out_pred", out_pred.get_shape())


    if hyper_param['use_decoder']:
        decoder_y_cat = Input(shape=(hyper_param['ner_classes'],), name='decoder_y_cat')
        masked_by_y = Mask(name='masked_by_y')([ner_caps, decoder_y_cat]) # true label is used to mask during training
        masked = Mask()(ner_caps) # mask using capsule with maximal length for predicion

        # decoder for training 
        train_decoder_dense1 = Dense(hyper_param['decoder_feed_forward_1'], activation='relu',\
                               input_dim=hyper_param['ner_capsule_dim']*hyper_param['ner_classes'],\
                               name='train_decoder_dense1')(masked_by_y)
        train_decoder_dense1_dropout = Dropout(hyper_param['decoder_dropout'])(train_decoder_dense1)
        train_decoder_dense2 = Dense(hyper_param['decoder_feed_forward_2'], activation='relu',\
                                     name='train_decoder_dense2')(train_decoder_dense1_dropout)
        train_decoder_dense2_dropout = Dropout(hyper_param['decoder_dropout'])(train_decoder_dense2)
        train_decoder_output = Dense(hyper_param['embed_dim'], activation='softmax',\
                                     name='train_decoder_output')(train_decoder_dense2_dropout)


        # decoder for evaluation (prediction) 
        eval_decoder_dense1 = Dense(hyper_param['decoder_feed_forward_1'], activation='relu',\
                               input_dim=hyper_param['ner_capsule_dim']*hyper_param['ner_classes'],\
                               name='eval_decoder_dense1')(masked)
        eval_decoder_dense2 = Dense(hyper_param['decoder_feed_forward_2'], activation='relu',\
                                     name='eval_decoder_dense2')(eval_decoder_dense1)
        eval_decoder_output = Dense(hyper_param['embed_dim'], activation='softmax',\
                                     name='eval_decoder_output')(eval_decoder_dense2)
        
        
        if verbose:
            print ("Decoder model enabled for GloVe vector deconstruction...")
            print ("decoder_y_cat", decoder_y_cat.get_shape())
            print ("masked_by_y", masked_by_y.get_shape())
            print ("train_decoder_dense1", train_decoder_dense1.get_shape())
            print ("train_decoder_dense1_dropout", train_decoder_dense1_dropout.get_shape())
            print ("train_decoder_dense2", train_decoder_dense2.get_shape())
            print ("train_decoder_dense2_dropout", train_decoder_dense2_dropout.get_shape())
            print ("train_decoder_output", train_decoder_output.get_shape())
            print ("masked", masked.get_shape())
            print ("eval_decoder_dense1", eval_decoder_dense1.get_shape())
            print ("eval_decoder_dense2", eval_decoder_dense2.get_shape())
            print ("eval_decoder_output", eval_decoder_output.get_shape())

    # construct input list
    if hyper_param['use_pos_tags'] and hyper_param['use_capitalization_info'] : 
        input_list = [x, x_pos, x_capital]
    elif hyper_param['use_pos_tags'] and (not hyper_param['use_capitalization_info']) :
        input_list = [x, x_pos]
    elif (not hyper_param['use_pos_tags']) and hyper_param['use_capitalization_info'] :
        input_list = [x, x_capital]
    else:
        input_list = [x]


    if hyper_param['use_decoder']==False:
        print ("decoder/reconstruction DISabled")
        print ("returning 1 model")
        return Model(inputs=input_list, outputs=[out_pred])
    else :
        train_model = Model(inputs=input_list+[decoder_y_cat], outputs=[out_pred, train_decoder_output])
        eval_model = Model(inputs=input_list, outputs=[out_pred, eval_decoder_output])
        print ("decoder/reconstruction enabled")
        print ("returning a list of 2 models: train_model, eval_model")
        return train_model, eval_model

### 3.1

In [ ]:
def draw_cnn_model(hyper_param, embedding_matrix=None ,verbose=True):
    """
    Input: hyper_parameters dictionary
    
    Construct:
        input layers : x , x_pos(o), x_captialization(o)
        embedding matrix : use_glove or randomly initialize
        conv1 : first convolution layer
        conv2 : second convolution layer
        conv3 : third convolution layer
        max pooling
        flatten : concant maxpooled univariate vectors into one long vector
        ff1, ff2: two feed forward layers
        out_pred: softmax over all ner classes
    
    Returns: keras.models.Model object
    """

    # input layer(s)
    x = Input(shape=(hyper_param['maxlen'],), name='x')
    if hyper_param['use_pos_tags'] : 
        x_pos = Input(shape=(hyper_param['maxlen'],hyper_param['poslen']), name='x_pos')
    if hyper_param['use_capitalization_info'] : 
        x_capital = Input(shape=(hyper_param['maxlen'], hyper_param['capitallen']), name='x_capital') 
    
    # embedding matrix
    if hyper_param['use_glove']:
        embed = Embedding(hyper_param['max_features'], hyper_param['embed_dim'], weights=[embedding_matrix],\
                          input_length=hyper_param['maxlen'], trainable=hyper_param['allow_glove_retrain'])(x)
    else:
        embed = Embedding(hyper_param['max_features'], hyper_param['embed_dim'], input_length=hyper_param['maxlen'],\
                          embeddings_initializer="random_uniform" )(x)

    # concat embeddings with additional features
    if hyper_param['use_pos_tags'] and hyper_param['use_capitalization_info'] : 
        embed = Concatenate(axis=-1)([embed, x_pos, x_capital])
    elif hyper_param['use_pos_tags'] and (not hyper_param['use_capitalization_info']) :
        embed = Concatenate(axis=-1)([embed, x_pos])
    elif (not hyper_param['use_pos_tags']) and hyper_param['use_capitalization_info'] :
        embed = Concatenate(axis=-1)([embed, x_capital])    
    else :
        embed = embed
    
    # feed embeddings into conv1
    conv1 = Conv1D( filters=hyper_param['conv1_filters'], \
                   kernel_size=hyper_param['conv1_kernel_size'],\
                   strides=hyper_param['conv1_strides'], \
                   padding=hyper_param['conv1_padding'],\
                   activation='relu', name='conv1')(embed)
    
    # update this
    # make primary capsules
    conv2 = Conv1D( filters=hyper_param['conv2_filters'], \
                   kernel_size=hyper_param['conv2_kernel_size'],\
                   strides=hyper_param['conv2_strides'], \
                   padding=hyper_param['conv2_padding'],\
                   activation='relu', name='conv2')(conv1)    
    
    # update this
    # make primary capsules
    conv3 = Conv1D( filters=hyper_param['conv3_filters'], \
                   kernel_size=hyper_param['conv3_kernel_size'],\
                   strides=hyper_param['conv3_strides'], \
                   padding=hyper_param['conv3_padding'],\
                   activation='relu', name='conv3')(conv2)    
    
    # max pooling layer
    max_pooled = MaxPooling1D(pool_size=hyper_param['max_pooling_size'], \
                              strides=hyper_param['max_pooling_strides'], \
                              padding=hyper_param['max_pooling_padding'])(conv3)
    # dropout
    maxpooled_dropout = Dropout(hyper_param['maxpool_dropout'])(max_pooled)
    
    # flatten many univariate vectos into 1 long vector
    flattened = Flatten()(maxpooled_dropout)
    
    # to feed-forward layers
    ff1 = Dense(hyper_param['feed_forward_1'], activation='relu')(flattened)    
    ff1_dropout = Dropout(hyper_param['ff1_dropout'])(ff1)
    
    ff2 = Dense(hyper_param['feed_forward_2'], activation='relu')(ff1_dropout)    
    ff2_dropout = Dropout(hyper_param['ff2_dropout'])(ff2)    
    
    out_pred = Dense(hyper_param['ner_classes'], activation='softmax', name='out_pred')(ff2) #!
    
             
    if verbose:
        print ("x", x.get_shape())
        if hyper_param['use_pos_tags'] : print ("x_pos", x_pos.get_shape())
        if hyper_param['use_capitalization_info'] : print ("x_capital", x_capital.get_shape())
        print ("embed", embed.get_shape())
        print ("embed", embed.get_shape())
        
        print ("conv1", conv1.get_shape())
        print ("conv2", conv2.get_shape())   
        print ("conv3", conv3.get_shape())
        print ("max_pooled", max_pooled.get_shape())
        print ("flattened", flattened.get_shape())   
        print ("ff1", ff1.get_shape())
        print ("ff2", ff2.get_shape())
        print ("out_pred", out_pred.get_shape())        
  
    # return final model
    if hyper_param['use_pos_tags'] and hyper_param['use_capitalization_info'] : 
        cnnmodel = Model(inputs=[x, x_pos, x_capital], outputs=[out_pred])
    elif hyper_param['use_pos_tags'] and (not hyper_param['use_capitalization_info']) :
        cnnmodel = Model(inputs=[x, x_pos], outputs=[out_pred])
    elif (not hyper_param['use_pos_tags']) and hyper_param['use_capitalization_info'] :
        cnnmodel = Model(inputs=[x, x_capital], outputs=[out_pred])   
    else :
        cnnmodel = Model(inputs=[x], outputs=[out_pred])
        
    return cnnmodel

### 3.2

In [14]:
def custom_cosine_proximity(y_true, y_pred):
    y_true = tf.nn.l2_normalize(y_true, dim=-1)
    y_pred = tf.nn.l2_normalize(y_pred, dim=-1)
    return -KB.sum(y_true * y_pred)

### 3.3

In [15]:
#margin_loss
def margin_loss(y_true, y_pred):
    L = y_true * KB.square(KB.maximum(0., 0.9 - y_pred)) + 0.5 * (1 - y_true) * KB.square(KB.maximum(0., y_pred - 0.1))
    return KB.mean(KB.sum(L, 1))

### 3.4

In [16]:
# !
def compile_caps_model(hyper_param, model): #!
    """
    Input: keras.models.Model object, see draw_capsnet_model() output. This is a graph with all layers drawn and connected
    
    do:
        compile with loss function and optimizer
    
    Returns: compiled model
    """
    if hyper_param['optimizer'] == "Adam":
        opt = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    elif hyper_param['optimizer'] == "SGD": 
        opt = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.5, nesterov=True)
    elif hyper_param['optimizer'] == None:
        raise Exception("No optimizer specified")

    if hyper_param.get('use_decoder') == True:
        model_loss = [margin_loss, custom_cosine_proximity] # work in progress
        loss_wts = [1, hyper_param['lam_recon']]
    else:
        model_loss = margin_loss
        loss_wts = None
    
    model.compile(optimizer=opt, #'adam',
                  loss=model_loss,
                  loss_weights=loss_wts,
                  metrics={'out_pred':'accuracy'})
    
    return model

### 3.5

In [17]:
def compile_cnn_model(hyper_param, model): #!
    """
    Input: keras.models.Model object, see draw_capsnet_model() output. This is a graph with all layers drawn and connected
    
    do:
        compile with loss function and optimizer
    
    Returns: compiled model
    """
    if hyper_param['optimizer'] == "Adam":
        opt = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    elif hyper_param['optimizer'] == "SGD": 
        opt = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.5, nesterov=True)
    elif hyper_param['optimizer'] == None:
        raise Exception("No optimizer specified")

    model.compile(optimizer=opt, #'adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

### 3.6

In [18]:
def fit_model(hyper_param, model, trainX_dict, devX_list_arrayS, trainY_dict, devY_list_arrayS ): #!
    #Saving weights and logging
    log = callbacks.CSVLogger(hyper_param['save_dir'] + '/log.csv')
    tb = callbacks.TensorBoard(log_dir=hyper_param['save_dir'] + '/tensorboard-logs', \
                               batch_size=hyper_param['batch_size'], histogram_freq=hyper_param['debug'])
    checkpoint = callbacks.ModelCheckpoint(hyper_param['save_dir'] + '/weights-{epoch:02d}.h5', \
                                           save_best_only=True, save_weights_only=True, verbose=1)
    #lr_decay = callbacks.LearningRateScheduler(schedule=lambda epoch: 0.001 * np.exp(-epoch / 10.))

    model.summary()
    #Save a png of the model shapes and flow
    #plot_model(capsmodel, to_file=save_dir + '/reuters-model.png', show_shapes=True)

    #loss = margin_loss
    
    data = model.fit( x=trainX_dict, # {'x':trainX, 'x_pos':trainX_pos_cat, 'x_capital':trainX_capitals_cat, (o)'decoder_y':trainY_cat}
                      y=trainY_dict, #!{'out_pred':trainY_cat, (o)'decoder_output':train_decoderY}
                      batch_size=hyper_param['batch_size'], 
                      epochs=hyper_param['epochs'], 
                      validation_data=[devX_list_arrayS, devY_list_arrayS], #! [devX, devX_pos_cat, devX_capitals_cat, (o)devY_cat], [devY_cat, (o)dev_decoderY]
                      callbacks=[log, tb, checkpoint], 
                      verbose=1)

### 4.0

In [19]:
#!
# define hyper parameters for model
# CAPSNET
hyper_param_caps = {
    
    'max_features' : vocabData.vocab.size,  # 20000
    'maxlen' : trainX.shape[1],  # window size (9)
    'poslen' : trainX_pos_cat.shape[2],  # pos classes (45)
    'capitallen' : trainX_capitals_cat.shape[2],  # capitalization classes (5)
    'ner_classes' : trainY_cat.shape[1],  # 8 
    'embed_dim' : embed_dim,  # word embedding size
    'num_routing' : 3, 

    'use_glove' : True,
    'allow_glove_retrain' : False,
    'use_pos_tags' : True,
    'use_capitalization_info' : True,    
    
    'conv1_filters' : 256,
    'conv1_kernel_size' : 3,
    'conv1_strides' : 1,
    'conv1_padding' : 'valid',
    
    'primarycaps_dim_capsule' : 8,
    'primarycaps_n_channels' : 32,
    'primarycaps_kernel_size' : 3,
    'primarycaps_strides' : 1,
    'primarycaps_padding' : 'valid',

    'ner_capsule_dim' : 16,
    
    'num_dynamic_routing_passes' : 3,
    
    # decoder is still work in progress
    'use_decoder' : False,
    'decoder_feed_forward_1' : 100,
    'decoder_feed_forward_2' : 100, 
    'decoder_dropout' : 0.3,
    
    'save_dir' : './result',
    'batch_size' : 100,
    'debug' : 2,
    'epochs' : 5,
    'dropout_p' : 0.25,
    'embed_dropout' : 0.25,
    'lam_recon' : 0.0005,
    
    'optimizer' : 'Adam', #or 'SGD'
}

### 4.1

In [20]:
#!
# define hyper parameters for model
# CNN
hyper_param_cnn = {
    
    'max_features' : vocabData.vocab.size,  # 20000
    'maxlen' : trainX.shape[1],  # window size (9)
    'poslen' : trainX_pos_cat.shape[2],  # pos classes (45)
    'capitallen' : trainX_capitals_cat.shape[2],  # capitalization classes (5)
    'ner_classes' : trainY_cat.shape[1],  # 8 
    'embed_dim' : embed_dim,  # word embedding size
    'num_routing' : 3, 

    'use_glove' : True,
    'allow_glove_retrain' : False,
    'use_pos_tags' : True,
    'use_capitalization_info' : True,    
    
    'conv1_filters' : 256,
    'conv1_kernel_size' : 3,
    'conv1_strides' : 1,
    'conv1_padding' : 'valid',
    
    'conv2_filters' : 256,
    'conv2_kernel_size' : 3,
    'conv2_strides' : 1,
    'conv2_padding' : 'valid',
    
    'conv3_filters' : 128,
    'conv3_kernel_size' : 3,
    'conv3_strides' : 1,
    'conv3_padding' : 'valid',
    
    'max_pooling_size' : 3,
    'max_pooling_strides' : 1,
    'max_pooling_padding' : 'valid',
    'maxpool_dropout' : 0.3,
    
    'feed_forward_1' : 328,
    'ff1_dropout' : 0.3,
    'feed_forward_2' : 192,
    'ff2_dropout' : 0.3,
    
    'save_dir' : './result',
    'batch_size' : 100,
    'debug' : 2,
    'epochs' : 5,
    'dropout_p' : 0.25,
    'embed_dropout' : 0.25,
    'lam_recon' : 0.0005,
    
    'optimizer' : 'Adam', #or 'SGD'
}

### 4.2

In [20]:
# no decoder
# draw and compile capsmodel GloVe embedding matrix
capsmodel = draw_capsnet_model(hyper_param=hyper_param_caps, embedding_matrix=embedding_matrix, verbose=True)
capsmodel = compile_caps_model(hyper_param_caps, capsmodel)

x (?, 9)
x_pos (?, 9, 45)
x_capital (?, 9, 5)
embed (?, 9, 100)
conv1 (?, 7, 256)
primarycaps (?, ?, 8)
ner_caps (?, 8, 16)
out_pred (?, 8)
decoder/reconstruction DISabled
returning 1 model


### 4.3

In [20]:
# draw and compile capsmodel GloVe embedding matrix
cnnmodel = draw_cnn_model(hyper_param=hyper_param_cnn, embedding_matrix=embedding_matrix, verbose=True)
cnnmodel = compile_cnn_model(hyper_param_cnn, cnnmodel)

x (?, 9)
x_pos (?, 9, 45)
x_capital (?, 9, 5)
embed (?, 9, 100)
embed (?, 9, 100)
conv1 (?, 7, 256)
conv2 (?, 5, 256)
conv3 (?, 3, 128)
max_pooled (?, 1, 128)
flattened (?, ?)
ff1 (?, 328)
ff2 (?, 192)
out_pred (?, 8)


### 4.4

In [54]:
# with decoder
capsmodel_decoder_train, capsmodel_decoder_eval = draw_capsnet_model(hyper_param=hyper_param_caps, embedding_matrix=embedding_matrix, verbose=True)
capsmodel_decoder = compile_caps_model(hyper_param_caps, capsmodel_decoder_train)

x (?, 9)
x_pos (?, 9, 45)
x_capital (?, 9, 5)
embed (?, 9, 100)
conv1 (?, 7, 256)
primarycaps (?, ?, 8)
ner_caps (?, 8, 16)
out_pred (?, 8)
Decoder model enabled for GloVe vector deconstruction...
decoder_y_cat (?, 8)
masked_by_y (?, ?)
train_decoder_dense1 (?, 100)
train_decoder_dense2 (?, 100)
train_decoder_output (?, 50)
masked (?, ?)
eval_decoder_dense1 (?, 100)
eval_decoder_dense2 (?, 100)
eval_decoder_output (?, 50)
decoder/reconstruction enabled
returning a list of 2 models: train_model, eval_model


### 4.5

In [21]:
fit_model(hyper_param_cnn, cnnmodel, \
          trainX_dict={'x':trainX, 'x_pos':trainX_pos_cat, 'x_capital':trainX_capitals_cat}, \
          devX_list_arrayS=[devX, devX_pos_cat, devX_capitals_cat], \
          trainY_dict={'out_pred':trainY_cat}, \
          devY_list_arrayS=[devY_cat] ) #!

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x (InputLayer)                  (None, 9)            0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 9, 50)        887400      x[0][0]                          
__________________________________________________________________________________________________
x_pos (InputLayer)              (None, 9, 45)        0                                            
__________________________________________________________________________________________________
x_capital (InputLayer)          (None, 9, 5)         0                                            
__________________________________________________________________________________________________
concatenat

### 4.6

In [21]:
fit_model(hyper_param_caps, capsmodel, \
          trainX_dict={'x':trainX, 'x_pos':trainX_pos_cat, 'x_capital':trainX_capitals_cat}, \
          devX_list_arrayS=[devX, devX_pos_cat, devX_capitals_cat], \
          trainY_dict={'out_pred':trainY_cat}, \
          devY_list_arrayS=[devY_cat] )

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x (InputLayer)                  (None, 9)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 9, 50)        887400      x[0][0]                          
__________________________________________________________________________________________________
x_pos (InputLayer)              (None, 9, 45)        0                                            
__________________________________________________________________________________________________
x_capital (InputLayer)          (None, 9, 5)         0                                            
__________________________________________________________________________________________________
concatenat

KeyboardInterrupt: 

### 4.7

In [56]:
# with decoder
fit_model(hyper_param_caps, capsmodel_decoder_train, \
          trainX_dict={'x':trainX, 'x_pos':trainX_pos_cat, 'x_capital':trainX_capitals_cat, 'decoder_y_cat':trainY_cat}, \
          devX_list_arrayS=[devX, devX_pos_cat, devX_capitals_cat, devY_cat], \
          trainY_dict={'out_pred':trainY_cat, 'train_decoder_output':train_decoderY}, \
          devY_list_arrayS=[devY_cat, dev_decoderY])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x (InputLayer)                  (None, 9)            0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (None, 9, 50)        887400      x[0][0]                          
__________________________________________________________________________________________________
x_pos (InputLayer)              (None, 9, 45)        0                                            
__________________________________________________________________________________________________
x_capital (InputLayer)          (None, 9, 5)         0                                            
__________________________________________________________________________________________________
concatenat

KeyboardInterrupt: 